# Word Embedding Visualizations.

Name: Anthony Kevin Oktavius

Here, we'll experiment with various embedding layer, with these following techniques.

* TF-IDF
* Count Vectorizer
* Word2Vec
* Neural network embedding layer.

As time passes, I'll try to add various  techniques. We also use dimensionality reduction technique to reduce the dimension of each layers, so we can visualize the dataset with matplotlib.

Library Import

In [9]:
import numpy as np
import zipfile
import gc
import pickle
import cv2
import math
import warnings
import random
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A
import torchvision
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import torch.utils.data as data_utils
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
# import torchmetrics
import torch.nn.functional as F
import PIL
import torch.utils.data as data_utils
import pickle
import collections

# import pytorch_lightning as pl
import imutils
import zipfile

from wordcloud import WordCloud
from PIL import Image
from torch.utils.data import Dataset
from collections import defaultdict
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python.client import device_lib
from zipfile import ZipFile
from IPython import display
from torchvision import models, transforms
from google.colab.patches import cv2_imshow
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.feature_extraction.text import CountVectorizer
from google.colab.patches import cv2_imshow

Dataset Loading with OpinRank dataset.

In [2]:
! wget https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz

--2023-09-28 09:56:58--  https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 443254787 (423M) [application/x-gzip]
Saving to: ‘enron_mail_20150507.tar.gz’

enron_mail_20150507 100%[===================>] 422.72M  4.36MB/s    in 98s     

2023-09-28 09:58:36 (4.33 MB/s) - ‘enron_mail_20150507.tar.gz’ saved [443254787/443254787]

unzip:  cannot find or open /content/enron_mail_20150507, /content/enron_mail_20150507.zip or /content/enron_mail_20150507.ZIP.


Unzip the dataset in tar file format.

In [8]:
! tar -xzf /content/enron_mail_20150507.tar.gz

## Data Preparation

Gather all of the email paths from all-documents folder.

In [13]:
root_path = "maildir/"
email_paths_from_all_documents_folder = glob.glob(os.path.join(root_path, "*/all_documents/*."))
print(email_paths_from_all_documents_folder[:10])

['maildir/whalley-g/all_documents/54.', 'maildir/whalley-g/all_documents/226.', 'maildir/whalley-g/all_documents/344.', 'maildir/whalley-g/all_documents/302.', 'maildir/whalley-g/all_documents/299.', 'maildir/whalley-g/all_documents/64.', 'maildir/whalley-g/all_documents/193.', 'maildir/whalley-g/all_documents/103.', 'maildir/whalley-g/all_documents/104.', 'maildir/whalley-g/all_documents/358.']


Get body email data, after X-Filename.

Performing Word Cloud.